In [ ]:
%run supportvectors-common.ipynb

# path to save training result plots. Set this variable before running the notebook.
dir_path = "data/results"

# Arithmetic with PyTorch


This is our first exercise with PyTorch. It comprises of writing a few simple classifiers and regressors. In particular, these models perform some arithmetic operations such determining the sum, standard-deviation, etc. for the sum, and determining if there is a negative number present in an array, or if the sum exceeds 100. A slightly more challenging task was to predict if any array contained a prime number.

**Caveat Emptor**

Remember, the purpose is **not to directly implement it as a PyTorch function, using the `torch` api directly** -- that
would be too simple. Instead, the goal is to train a neural network to learn the underlying functions that can do this.

Since neural networks are universal approximators, we therefore have to cast it into a form such that:
$$ \hat{y} = f(x) $$
where $f(\cdot)$ does the arithmetic operation (or an approximation of it).

    
## Homework assignment details

The specific details of the homework assignment are here:

Let us build a few classifier and regressor models from scratch. In particular, let us build the following models:

### Classifiers
* A model to detect if there is a negative number present in a set of 10 numbers as input. Output the result as a probability, and also a label, such as "True", "False"
* A model to predict if the sum of the input numbers (10 of these) adds up to greater than 100. Once again, output a probability, and a label.
* (Stretch goal) A model to detect the presence of a prime number in a set of 10 numbers given. Assume the numbers are less than a million.

### Regressors
* A model to find the sum of inputs (10 numbers as input)
* A model to find the standard deviation of the inputs (10 numbers as input)
* A model to find the maximum in a set of 10 numbers as input

In order to train these models, you will have to do a careful generation of training data.

In [2]:
# imports
import numpy as np
from rich import print as rprint

from svlearn.data.numpy_dataset_generator import NumpyDatasetGenerator
from svlearn.nn.arithmetic_negative_number_detector import NegativeNumberDetector
from svlearn.data.simple_torch_dataset import SimpleNumpyDataset
from svlearn.train.classification_trainer import ClassificationTrainer
from svlearn.train.regression_trainer import RegressionTrainer
from svlearn.train.visualization_utils import visualize_classification_training_results , visualize_regression_training_results


# torch
import torch
from torch import nn
from torch.nn import BCELoss , MSELoss, CrossEntropyLoss
from torch.optim.lr_scheduler import StepLR
from svlearn.train.fit import ModelFitter
from torch.optim import AdamW

## Summation of the input numbers

Let us perform the following tasks

 1. Generate the data
 2. Instantiate the regressor
 3. Train and evaluate the regressor with the generated data
 4. Visualize the training results


### Create a PyTorch dataset
First let's create datasets for training and evaluation. `NumpyDatasetGenerator` generates all the datasets for this exercise. It creates both the samples `X` and the labels `y`associated with each sample  

In [3]:
num_features = 10

# create a numpy dataset for training
train_dataset_generator = NumpyDatasetGenerator(num_samples=1000 , num_features=num_features)
X , y = train_dataset_generator.generate_sum_features_dataset()

# create a pytorch dataset from the numpy arrays
train_dataset = SimpleNumpyDataset(X , y)

# create a numpy dataset for evaluation
eval_dataset_generator = NumpyDatasetGenerator(num_samples=300 , num_features=num_features)
X , y = eval_dataset_generator.generate_sum_features_dataset()

# create a pytorch dataset from the numpy arrays
eval_dataset = SimpleNumpyDataset(X , y)

Let's take a sample from the dataset 

In [ ]:
train_dataset[0]

We get the 10 numbers as a pytorch array along with the label associated with the sample.

### Instantiate the Regressor

Next, let's decide on how our neural network should be designed so that when provided with a sample of 10 numbers, it returns their sum. How should we go about this problem? 

Let's determine the inputs and outputs of model.

the model needs to map 10 values to 1 , i.e. it's sum. 
So the input dimension is 10 and the output dimension is 1. 

Next, what transformation should we apply ? 

$$ x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9 + x_{10} = s $$

We can rewrite the above as a linear transformation applied on a vector x as shown below

$$
\begin{bmatrix}
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\
x_6 \\
x_7 \\
x_8 \\
x_9 \\
x_{10}
\end{bmatrix}
 + 0
= s $$

$$
\mathbf{w}^T \cdot \mathbf{x} + b = s
$$

From this, we can conclude that a single linear transformation is sufficient to solve this problem. Note that no non-linear activation function is required at all! Now let's review the neural architecture of the `SumRegressor`

### Instantiate model and trainer

In [ ]:
from svlearn.nn.arithmetic_sum_regressor import SumRegressor

model = SumRegressor(input_dim=10)

# print initial weights
print("Model's state dict before training:")
for param_tensor in model.state_dict():
    rprint(param_tensor, "\t", model.state_dict()[param_tensor])

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Trainer instance
trainer = RegressionTrainer(
    train_dataset=train_dataset,  
    eval_dataset=eval_dataset,
    model=model,
    loss_func=MSELoss(),
    optimizer=optimizer,
    num_epochs=100,
    batch_size=32,
    device='cuda',
    show_every=50
)

Now let's train the model and visualize the results

In [ ]:
trainer.train()

In [ ]:
metrics = trainer.metrics_dict 
visualize_regression_training_results(metrics['train_loss'] , 
                                          metrics['eval_loss'] , 
                                          dir_path=dir_path , 
                                          filename="sum_regressor")

As expected the loss converges to 0 very rapidly. Let's view the weights of the model after the training. Have the weights converged to what we assumed it would?

Note that we built the neural network with a single neuron -- we have only a linear operation to perform, namely the summation. Therefore, we use a single neuron, and eschew the activation function altogether, since we have no need for a deformation or non-linearity!

In [ ]:
# print initial weights
print("Model's state_dict after training:")
for param_tensor in model.state_dict():
    rprint(param_tensor, "\t", model.state_dict()[param_tensor])

## Detect negative numbers

What about detecting the negative numbers in an array? Can we do that with a single neuron? Do we need an activation function ?



### Create dataset
Let's first create a dataset by randomly sampling a normal distribution making sure that a signification portion of the distribution includes negative values.



In [9]:
num_features = 10

# create a numpy dataset for training
train_dataset_generator = NumpyDatasetGenerator(num_samples=1000 , num_features=num_features)
X , y = train_dataset_generator.generate_negative_number_dataset(mean=50 , std=25)

# create a pytorch dataset from the numpy arrays
train_dataset = SimpleNumpyDataset(X , y)

# create a numpy dataset for evaluation
eval_dataset_generator = NumpyDatasetGenerator(num_samples=300 , num_features=num_features)
X , y = eval_dataset_generator.generate_negative_number_dataset(mean=50 , std=25)

# create a pytorch dataset from the numpy arrays
eval_dataset = SimpleNumpyDataset(X , y)

### Instantiate model and trainer

Let us think about the architecture for this network, and in particular, think about the activation function `ReLu` (Rectified Linear Unit). 

Recall that it turns off for all negative numbers, but leaves the positive values unaltered. 

In [10]:
model = NegativeNumberDetector(input_dim=num_features)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Trainer instance
trainer = ClassificationTrainer(
    train_dataset=train_dataset, 
    eval_dataset=eval_dataset, 
    model=model,
    loss_func=BCELoss(),
    optimizer=optimizer,
    num_epochs=200,
    batch_size=32,
    device='cuda',
    show_every=50
)

In [ ]:
trainer.train()

### Visualize results

In [ ]:
metrics = trainer.metrics 
visualize_classification_training_results(metrics['train_loss'] , 
                                          metrics['eval_loss'] , 
                                          metrics['train_acc'] , 
                                          metrics['eval_acc'], 
                                          dir_path=dir_path, 
                                          filename="negative_numbers_detector")

Note that for this problem we had to introduce non-linearity in addition to a linear transformation. 

## Detect sum greater than 100

This is essentially the same as the first  problem, recast as a binary classification problem.

### Create dataset

In [13]:
num_features = 10

train_dataset_generator = NumpyDatasetGenerator(num_samples=1000 , num_features=num_features)
X , y = train_dataset_generator.generate_sum_greater_than_100_dataset()
train_dataset = SimpleNumpyDataset(X , y)

eval_dataset_generator = NumpyDatasetGenerator(num_samples=300 , num_features=num_features)
X , y = eval_dataset_generator.generate_sum_greater_than_100_dataset()
eval_dataset = SimpleNumpyDataset(X , y)

### Instantiate model and trainer

In [ ]:
from svlearn.nn.arithmetic_sum_greater_than_detector import SumGreaterThan100Detector

# Assume train_dataset and eval_dataset are instances of NumpyDataset
input_dim = 10
model = SumGreaterThan100Detector(input_dim=input_dim)

# model.apply(initialize_weights)

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor])

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Trainer instance
trainer = ClassificationTrainer(
    train_dataset=train_dataset,  
    eval_dataset=eval_dataset,
    model=model,
    loss_func=BCELoss(),
    optimizer=optimizer,
    num_epochs=1000,
    batch_size=64,
    device='cuda',
    show_every=50
)

In [ ]:

trainer.train()

In [ ]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor])

In [ ]:
metrics = trainer.metrics 
visualize_classification_training_results(metrics['train_loss'] , 
                                          metrics['eval_loss'] , 
                                          metrics['train_acc'] , 
                                          metrics['eval_acc'], 
                                          dir_path=dir_path , 
                                          filename="sum_greater_than_100_detector")

## Standard Deviation Regressor

 Variance (square of std) has quadratic terms, and thus is not linear in inputs. Therefore, you will need deformations to get higher order terms...and you will need more than 1 neuron for the same reason (to get both the quadratic and linear terms separately). You can convince yourself that a simple, small 2 layer-network can solve this, and empirically verify.

 We are instead going to deliberately take a more complex network to show that it converges much faster, so long as you have enough training data.

In [18]:
num_features = 10

train_dataset_generator = NumpyDatasetGenerator(num_samples=10000 , num_features=num_features)
X , y = train_dataset_generator.generate_variable_std_dataset(mean=5)
train_dataset = SimpleNumpyDataset(X , y)

eval_dataset_generator = NumpyDatasetGenerator(num_samples=3000 , num_features=num_features)
X , y = eval_dataset_generator.generate_variable_std_dataset(mean=5)
eval_dataset = SimpleNumpyDataset(X , y)

In [19]:
from svlearn.nn.arithmetic_std_regressor import StdRegressor
from torch.optim.lr_scheduler import StepLR

scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

# Assume train_dataset and eval_dataset are instances of NumpyDataset
input_dim = 10
model = StdRegressor(input_dim=input_dim)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

# Trainer instance
trainer = RegressionTrainer(
    train_dataset=train_dataset,  
    eval_dataset=eval_dataset,
    model=model,
    loss_func=MSELoss(),
    optimizer=optimizer,
    num_epochs=300,
    batch_size=1024,
    device='cuda',
    scheduler=scheduler,
    show_every=50
)

In [ ]:
trainer.train()

In [ ]:
metrics = trainer.metrics_dict 
visualize_regression_training_results(metrics['train_loss'] , 
                                          metrics['eval_loss'] , 
                                          dir_path=dir_path , 
                                          filename="std_regressor")

## Max classifier

Finally, for this exercise we do not need a neural network at all! Simply applying the softmax function will convert the list of numbers into probabilities. 

In [22]:
num_features = 10

train_dataset_generator = NumpyDatasetGenerator(num_samples=1000 , num_features=num_features)
X , y = train_dataset_generator.generate_max_index_labels_dataset()
train_dataset = SimpleNumpyDataset(X , y)

sample_input , sample_target = train_dataset[0]

In [ ]:
sample_input

In [ ]:
sample_target

In [ ]:
from torch.nn import Softmax

torch.argmax(Softmax()(sample_input)) # prediction

## Prime number detector

This is a much harder problem; but let us reason through it. There are 78,498 primes below 1 million. Therefore, if we still only to an array or positive numbers, the probability that a number is prime and below a million would be: 


In [ ]:
p = 78_498/1_000_000
p

In [ ]:
# Probability none of the 10 numbers being prime therefore is:

no_primes = (1-p)**10
no_primes

This, therefore, forms the lower bound on the accuracy of any classifier that detects primes. If the detector just blindly says that there is at-least one prime in the array, it would be right about 56% of the times.So a good classifier must beat a performance level of **56%** of accuracy for prime detection.

But here we meet the inherent limitation of neural networks -- since there is no known continuous function that can generate the primes, therefore one cannot assume that (because of the Universal function approximation theorem,) a neural network will necessarily succeed in modeling primes.

Indeed, the instructor gave this problem specifically to illustrate the current limits of what a neural network can do.

## Factorial approximator?

Let us ask this question -- can we have a neural network that can compute factorials? One would wonder -- how will we get training data for it, since factorials blow up exponentially, and quickly exceed the capacity of `float32` ? And we can train a network with a handful of few integers whose factorials are feasible.

Fortunately, there is a generalization of factorial to the real-valued field: it is the $\Gamma (x)$ function; therefore, we can generate an enormous amount to labeled training data:

The Gamma function, $\Gamma (x)$, is defined for all complex numbers except the non-positive integers. For any positive number $x$, it is defined as an integral from $0$ to $\infty$:

$$ \Gamma(x) = \int_{0}^{\infty} t^{x-1} e^{-t} \, dt $$

This integral converges for $x > 0$. The Gamma function extends the concept of factorial (normally defined only for non-negative integers) to real and complex numbers, except for non-positive integers where the function is not defined. For positive integers $n$, it satisfies the relationship:

$$ \Gamma(n) = (n-1)! $$

This means, for example, $\Gamma(5) = 4!\ = 24$, which extends the factorial notion to non-integer values smoothly. For non-integer values, the Gamma function can be computed using this integral definition or various numerical approximations and series expansions.


In [28]:
from scipy.special import gamma

def generate_gamma_data(n: int = 100_000):

    x = np.linspace(1, 22, n).reshape(-1, 1)
    y = gamma(x).reshape(-1, 1)
    return x-1, y

In [ ]:
# Let us first verify that it works
x, y = generate_gamma_data(22)
for xx, yy in zip(x, y):
    print(xx, yy)

As we clearly see, it is working. We can verify that with $10!$:

In [ ]:
import math
math.factorial(10)

Therefore, let us train a neural network for this, and see what happens; read the associated neural network in the `svlearn.nn` module to see the simple feed-forward network we built for this:

In [ ]:
from svlearn.nn.multilayer_regressor import MultilayerRegressor
from scipy.special import gamma
from skorch import NeuralNetRegressor


def generate_gamma_data(n: int = 100):

    np.random.seed(0)
    x = 1+ np.linspace(0, 5, n)
    np.random.shuffle(x)
 
    y = gamma(x)
    
    y=y.reshape(-1, 1).astype(np.float32)
    x = (x-1).reshape(-1, 1).astype(np.float32)
    return x, y

regressor = NeuralNetRegressor(
    MultilayerRegressor,
    max_epochs=50,
    lr=0.001,
    batch_size=1024,
    optimizer=AdamW,
    criterion=nn.MSELoss,
    module__input_dimension=1,
    module__output_dimension=1,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# Create the training data
from svlearn.arithmetic.data import ArithmeticData
x_train, y_train = generate_gamma_data(100_000)
x_test, y_test = generate_gamma_data(100_000)

ModelFitter().regression_fit_evaluate_plot(regressor, x_train, y_train, x_test, y_test)

You will notice that with a smaller range, up to 5, in this case, the predictions are very good. However, as you increase the range of data, you will have to make quite some effort to think through your training data distribution, before you get better results, and you will have to train for much longer. This is primarily because of the exponential nature of the factorial. 